<a href="https://colab.research.google.com/github/Abdullah-Al-Arafat/CAP-5610-Machine-Learning/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAP 5610: Machine Learning** 
Home work 2


In [0]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad, Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation, Flatten, BatchNormalization
from keras import regularizers

Preparation/preprocessing of train, test, and validation dataset 

In [0]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:10000]
train_indices = indices[10000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

**Model without Dropout layers and Data Augmentation** 

In [4]:
model1 = Sequential()
model1.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))


model1.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
#model1.add(Dropout(0.5))
 
model1.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())
#model.add(BatchNormalization())
#model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))
model1.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model1.add(Dense(64, activation= 'relu'))
model1.add(Dense(10, activation='softmax'))

model1.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 48)        13872     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 2, 256)         295168    
__________

In [5]:
# Initiate Adagrad/RMSprop optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model1.fit(X_train, y_train, batch_size= 40, epochs= 20,
                    verbose=1, validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 30s 747us/step - loss: 1.6334 - acc: 0.4229 - val_loss: 1.3277 - val_acc: 0.5542
Epoch 2/20
40000/40000 [==============================] - 24s 597us/step - loss: 1.1328 - acc: 0.6221 - val_loss: 1.1045 - val_acc: 0.6405
Epoch 3/20
40000/40000 [==============================] - 24s 599us/step - loss: 0.9275 - acc: 0.7012 - val_loss: 0.9564 - val_acc: 0.6899
Epoch 4/20
40000/40000 [==============================] - 24s 599us/step - loss: 0.7964 - acc: 0.7503 - val_loss: 0.9246 - val_acc: 0.7060
Epoch 5/20
40000/40000 [==============================] - 24s 596us/step - loss: 0.6981 - acc: 0.7845 - val_loss: 0.9039 - val_acc: 0.7158
Epoch 6/20
40000/40000 [==============================] - 24s 597us/step - loss: 0.6157 - acc: 0.8159 - val_loss: 0.8941 - val_acc: 0.7296
Epoch 7/20
40000/40000 [==============================] - 24s 600us/

In [6]:
## Score trained model.
scores = model1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 228us/step
Test loss: 1.8532806515693665
Test accuracy: 0.7291


model1 is clearly overfitting the training dataset, althought I have added L1 and L2 regularization in the Conv layers and FC layers.

**Model2** with dropout layers

In [7]:
model2 = Sequential()
model2.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))


model2.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model1.add(Dropout(0.5))
 
model2.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model2.add(Dropout(0.25))

model2.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))

model2.add(Flatten())
model2.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model2.add(Dense(64, activation= 'relu'))
model2.add(Dense(10, activation='softmax'))

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 48)        13872     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 128)         73856     
__________

In [8]:
# Initiate Adagrad/RMSprop optimizer
model2.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model2.fit(X_train, y_train, batch_size= 40, epochs= 25,
                    verbose=1, validation_data=(X_val, y_val))

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 29s 728us/step - loss: 1.8006 - acc: 0.3723 - val_loss: 1.4672 - val_acc: 0.4963
Epoch 2/25
40000/40000 [==============================] - 29s 713us/step - loss: 1.3368 - acc: 0.5496 - val_loss: 1.1922 - val_acc: 0.6120
Epoch 3/25
40000/40000 [==============================] - 29s 719us/step - loss: 1.1355 - acc: 0.6287 - val_loss: 1.0844 - val_acc: 0.6494
Epoch 4/25
40000/40000 [==============================] - 29s 715us/step - loss: 1.0247 - acc: 0.6680 - val_loss: 1.0113 - val_acc: 0.6760
Epoch 5/25
40000/40000 [==============================] - 29s 714us/step - loss: 0.9456 - acc: 0.6974 - val_loss: 0.9491 - val_acc: 0.6994
Epoch 6/25
40000/40000 [==============================] - 28s 708us/step - loss: 0.8752 - acc: 0.7240 - val_loss: 0.9116 - val_acc: 0.7182
Epoch 7/25
40000/40000 [==============================] - 28s 712us/step - loss: 0.8265 - acc: 0.7424 - val_loss: 0.

In [9]:
## Score trained model.
scores = model2.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 222us/step
Test loss: 0.8324978959083558
Test accuracy: 0.771


**Model2** significantly reduce the varince, now I'll introduce data augmentation in the model3

In [0]:
#data augmentation

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=60,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

In [0]:
model3 = Sequential()
model3.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))


model3.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model3.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
 
model3.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model3.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model3.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))

model3.add(Flatten())
model3.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model3.add(Dense(64, activation= 'relu'))
model3.add(Dense(10, activation='softmax'))

In [23]:
model3.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])



## Train the model and get the training and validation loss
model3.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs=25,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/25
1250/1250 [==============================] - 48s 39ms/step - loss: 1.8915 - acc: 0.3369 - val_loss: 1.5144 - val_acc: 0.4639
Epoch 2/25
1250/1250 [==============================] - 47s 37ms/step - loss: 1.5803 - acc: 0.4527 - val_loss: 1.3523 - val_acc: 0.5321
Epoch 3/25
1250/1250 [==============================] - 46s 37ms/step - loss: 1.4345 - acc: 0.5069 - val_loss: 1.2867 - val_acc: 0.5626
Epoch 4/25
1250/1250 [==============================] - 46s 37ms/step - loss: 1.3454 - acc: 0.5396 - val_loss: 1.2896 - val_acc: 0.5708
Epoch 5/25
1250/1250 [==============================] - 45s 36ms/step - loss: 1.2755 - acc: 0.5678 - val_loss: 1.1599 - val_acc: 0.6132
Epoch 6/25
1250/1250 [==============================] - 45s 36ms/step - loss: 1.2386 - acc: 0.5834 - val_loss: 1.1522 - val_acc: 0.6205
Epoch 7/25
1250/1250 [==============================] - 45s 36ms/step - loss: 1.1902 - acc: 0.6032 - val_loss: 1.1660 - val_acc: 0.6133
Epoch 8/25
1250/1250 [==========================

In [24]:
## Score trained model.
scores = model3.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 215us/step
Test loss: 0.956143762588501
Test accuracy: 0.6998


**Moel4**  Dropout with Data augmentation

In [0]:
model4 = Sequential()
model4.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))


model4.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.25))

model4.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model4.add(Dropout(0.5))
 
model4.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model4.add(Dropout(0.25))

model4.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model4.add(Dropout(0.1))

model4.add(Flatten())
model4.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model4.add(Dense(64, activation= 'relu'))
model4.add(Dense(10, activation='softmax'))

In [26]:
model4.compile(loss='categorical_crossentropy', 
               optimizer='adagrad', metrics=['accuracy'])



## Train the model and get the training and validation loss
model4.fit_generator(datagen.flow(X_train, y_train, batch_size=40),
                    steps_per_epoch=train_images.shape[0]/ 40,epochs=25,
                    verbose=1,validation_data=(X_val,y_val))

Epoch 1/25
1250/1250 [==============================] - 49s 39ms/step - loss: 2.0661 - acc: 0.2803 - val_loss: 1.8646 - val_acc: 0.3600
Epoch 2/25
1250/1250 [==============================] - 47s 38ms/step - loss: 1.8413 - acc: 0.3598 - val_loss: 1.6900 - val_acc: 0.4340
Epoch 3/25
1250/1250 [==============================] - 47s 38ms/step - loss: 1.7237 - acc: 0.4018 - val_loss: 1.5719 - val_acc: 0.4570
Epoch 4/25
1250/1250 [==============================] - 47s 38ms/step - loss: 1.6519 - acc: 0.4292 - val_loss: 1.5319 - val_acc: 0.4704
Epoch 5/25
1250/1250 [==============================] - 48s 38ms/step - loss: 1.5892 - acc: 0.4514 - val_loss: 1.4206 - val_acc: 0.5098
Epoch 6/25
1250/1250 [==============================] - 47s 38ms/step - loss: 1.5497 - acc: 0.4689 - val_loss: 1.4257 - val_acc: 0.5089
Epoch 7/25
1250/1250 [==============================] - 47s 37ms/step - loss: 1.5116 - acc: 0.4825 - val_loss: 1.4589 - val_acc: 0.4928
Epoch 8/25
1250/1250 [==========================

For k-fold validation, I have used the best model from above

In [27]:
## Get the training dataset again

from sklearn.model_selection import KFold

X_train = train_images.reshape(train_images.shape[0], 32, 32, 3)
y_train_Kfold = np_utils.to_categorical(train_labels_1, nb_classes)
X_train_Kfold = (X_train/255).astype('float32')


scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X_train_Kfold):
  
    ## K-fold Split of the dataset(where K=20, 19 of the folds are for training and 1 of them is for validating)
    X_train, X_val, y_train, y_val = X_train_Kfold[train_index], X_train_Kfold[test_index], y_train_Kfold[train_index], y_train_Kfold[test_index]

    ## Compile the optimizer and train 
    model4.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    model4.fit(X_train, y_train)

    ## Get the score on the test dataset
    scores.append(model4.evaluate(X_test, y_test))
    
print("K-fold cross validation DONE...")
print("The Cross validation Score [Accuracy]",np.mean(scores[1]))

NameError: ignored